In [8]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy.stats as sci
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



***
### Data Import


In [24]:
test = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/test_sample.csv")
train = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/train_sample.csv")

# test = pd.get_dummies(test)
# train = pd.get_dummies(train)

In [ ]:
# import pandas as pd
# from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.inspection import permutation_importance
# from sklearn.feature_selection import SelectFromModel



# train = pd.get_dummies(train)
# test = pd.get_dummies(test)


# # separate predictor column from train dataset
# X_train = train.drop(columns=['isFraud'])
# y_train = train['isFraud']

# # load test dataset
# # test_df = pd.read_csv("test.csv")
# # create an instance of RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=100, random_state=0)

# # create a SelectFromModel transformer with RF as the base estimator
# sfm = SelectFromModel(rf, max_features=50)

# # fit SFM on train dataset
# sfm.fit(X_train, y_train)

# # transform train and test datasets with selected features
# X_train_selected = sfm.transform(X_train)
# X_test_selected = sfm.transform(test_df)


***
### Misc Functions

In [ ]:
# Cleaning the columns with T/F and making them 1/0
for i in range(1,10):
    train['M'+str(i)] = train['M'+str(i)].replace({'T': 1, 'F': 0})

In [ ]:
# Some code I found that will go through and mark the unique entries in each column, giving the general estimate of what each does
for col, values in train.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

In [ ]:
dtypething = {k:v for k, v in (train.dtypes == 'O').to_dict().items() if v is True}


for k, v in dtypething.items():
    train[k] = train[k].astype(str)

***
### Distribution Graphs


In [ ]:
transactions = train[train['TransactionAmt'] >0 ]['TransactionAmt']

# Dist of TransactionAmt
fig = px.histogram(
    transactions, 
    title= "Distribuiton of Transactions",
    nbins=1000
    ) 

fig.update_layout( height=500, width=1000)
fig.update_xaxes(range = [0,5000])

In [ ]:
transactions = train[train['TransactionAmt'] > 0 ]['TransactionAmt']

# Dist of TransactionAmt
fig = px.box(
    transactions, 
    title= "Distribuiton of Transactions"
    ) 

fig.update_layout( height=500, width=750)
# fig.update_xaxes(range = [0,5000])
fig.show()

In [ ]:
Card_Dist = pd.value_counts(train['card4'],normalize= True, dropna = False)

# Dist of Card Types
fig = px.bar( 
    data_frame = Card_Dist,
    title='Distribution of Card Type',color = Card_Dist.keys(),
    text= Card_Dist.apply(lambda x: f'{x*100:.2f}%'),
    orientation='h'
    )

fig.update_layout(xaxis={'title': 'Card Types'},yaxis={'title': 'Percentage'}, height=500, width=1000)

In [ ]:
dist = pd.value_counts(train['DeviceInfo'],normalize= True, dropna = False)

amount_to_show = 9
new_data = {}

for i, (key, value) in enumerate(dist.to_dict().items()):
    if i < amount_to_show:
        new_data[key] = value
    else:
        new_data['Other'] = new_data.get('Other', 0) + value

fig = px.pie(
    values=list(new_data.values()), 
    names=list(new_data.keys()),
    title= 'Distribution Of Operating System That Made The Transaction'
    )

fig.update_traces(textinfo='percent+label')
fig.show()

***
### Correlation Stuff

In [ ]:
train_corr = train.corr()

In [ ]:


# Extract the correlations as a DataFrame
abscorr = train_corr.abs()
# Filter the DataFrame to show only correlations greater than 0.7 and not equal to 1
high_corr = abscorr[(abscorr > 0) & (abscorr != 1)]
# Print the high correlation pairs
high_corr_dict = (high_corr.stack().drop_duplicates()).to_dict()

# Filter the high correlation pairs by 'isFraud' and any column that starts with 'C'
corrs = {k:v for k, v in high_corr_dict.items() if ('isFraud' in k) and any(col.startswith('C') for col in k)}

# Sort the pairs by correlation value in descending order
sorted_corrs = sorted(corrs.items(), key=lambda x: x[1], reverse=True)

# Get the column with the highest correlation with 'isFraud'
highest_corr_column = sorted_corrs[0][0][1:]

print("The column with the highest correlation with 'isFraud' is:", highest_corr_column)


***
### Confidence Interval Stuff

In [ ]:
proplist = [1,2,3,5,6,7,8,9]
card_props = []
for i in proplist:
    prop = train['M'+str(i)].mean()
    card_props.append(prop)
total_card = len(card_props)

for i in card_props:
    se = np.sqrt(i*(1-i)/total_card)
    me = sci.t.ppf(0.975, total_card-1)*se
    ci = (i-me, i+me)
    print('=======================================')
    print('Sample Proportion:', i)
    print('Confidence Interval (95%):', ci)


***
### Random Forest Stuff

Our top 10 vars!


In [ ]:
from sklearn.impute import SimpleImputer

supertrainset = train.copy()
# supertrainset.drop(columns=['C1','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13'], inplace=True)

yass = supertrainset.reset_index(drop=True)  # update here
imputer = SimpleImputer(strategy='most_frequent')
supertrainset = pd.DataFrame(imputer.fit_transform(yass), columns=yass.columns)


In [ ]:
X = supertrainset.drop(['isFraud'],axis = 1)
Y = supertrainset['isFraud']
Y = LabelEncoder().fit_transform(Y)
X2 = StandardScaler().fit_transform(X)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X2,Y,test_size = 0.3, random_state = 101)

trainedForest = RandomForestClassifier(n_estimators = 700).fit(X_Train,Y_Train)
prediciton = trainedForest.predict(X_Test)

In [ ]:
feat_importances = pd.Series(trainedForest.feature_importances_, index = X.columns)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
importance = feat_importances.nlargest(10).keys().to_list()

X_Reduced = X[importance]
X_Reduced = StandardScaler().fit_transform(X_Reduced)
X_Train2, X_Test2, Y_Train2, Y_Test2 = train_test_split(X_Reduced, Y, test_size = 0.30, random_state = 101)


trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train2,Y_Train2)
predictionforest = trainedforest.predict(X_Test2)

VERY SLIGHT DROP IN PREFORMANCE

In [ ]:
print('================================================================')
print('All Features')
print(classification_report(Y_Test,prediciton))
print('================================================================')
print('Top 10 Features')
print(classification_report(Y_Test2,predictionforest))
print('================================================================')

***